StyleGAN2-ADA Fine-tune & Generate for FER2013 (Class: **disgust**)


This Colab notebook automates the process of:
1. Installing **StyleGAN2-ADA (NVLabs)**
2. Mounting Google Drive
3. Extracting **“disgust”** images from `fer2013.csv`
4. Resizing to **256×256**
5. Creating dataset `.zip` for StyleGAN2
6. Fine-tuning from pretrained **FFHQ** weights
7. Generating new synthetic images
8. *(Optional)* Calculating **FID**

In [ ]:
## ⚙️ SECTION 0 — CONFIGURATION

DRIVE_MOUNT_POINT = '/content/drive'
FER_CSV_DRIVE_PATH = '/content/drive/MyDrive/Work_space/project_facial_expression/stylegan2ada/dataset_fer2013/Copy of fer2013.csv' # Update if needed
PROJECT_DIR = '/content/drive/MyDrive/stylegan2_fer2013' # Checkpoints and outputs
WORK_DIR = '/content/stylegan2-ada-pytorch'
IMG_RESOLUTION = 256
TARGET_CLASS_LABEL = 1 # 0=angry, 1=disgust, 2=fear, 3=happy, 4=sad, 5=surprise, 6=neutral
SAMPLES_TO_GENERATE = 500
BATCH_SIZE = 8
GPUS = 1


In [ ]:
# Check GPU
!nvidia-smi

# Clone repo chính thức của StyleGAN2-ADA-PyTorch
!git clone --depth=1 https://github.com/NVlabs/stylegan2-ada-pytorch.git
%cd stylegan2-ada-pytorch

# Cài đặt các dependency cần thiết
!pip install ninja opensimplex==0.3 protobuf==3.20.3
!pip install imageio-ffmpeg==0.4.3

# Kiểm tra môi trường torch và CUDA
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))



Thu Oct  9 05:57:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 75.9 MB/s eta 0:00:00
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.6.0
    Uninstalling imageio-ffmpeg-0.6.0:
      Successfully uninstalled imageio-ffmpeg-0.6.0
Torch version: 2.8.0+cu126
CUDA available: True
GPU name: Tesla T4


In [ ]:
## 💾 SECTION 2 — MOUNT GOOGLE DRIVE

from google.colab import drive
drive.mount(DRIVE_MOUNT_POINT)


Mounted at /content/drive


In [ ]:
## 🧠 SECTION 3 — EXTRACT DISGUST IMAGES FROM `fer2013.csv`

import os, pandas as pd, numpy as np, cv2
from tqdm import tqdm


os.makedirs('/content/fer2013/disgust', exist_ok=True)


df = pd.read_csv(FER_CSV_DRIVE_PATH)
# Filter only disgust (label 1)
disgust_df = df[df['emotion'] == 1]


for i, row in tqdm(disgust_df.iterrows(), total=len(disgust_df)):
  pixels = np.array(row['pixels'].split(), dtype='uint8').reshape(48,48)
  cv2.imwrite(f"/content/fer2013/disgust/{i}.png", pixels)


100%|██████████| 547/547 [00:00<00:00, 1555.36it/s]


In [ ]:
## 🖼️ SECTION 4 — RESIZE IMAGES TO 256x256

import glob
os.makedirs('/content/fer2013_256/disgust', exist_ok=True)


for img_path in tqdm(glob.glob('/content/fer2013/disgust/*.png')):
  img = cv2.imread(img_path)
  resized = cv2.resize(img, (IMG_RESOLUTION, IMG_RESOLUTION))
  out_path = img_path.replace('fer2013', 'fer2013_256')
  cv2.imwrite(out_path, resized)


100%|██████████| 547/547 [00:01<00:00, 360.43it/s]


In [ ]:
## 📦 SECTION 5 — CREATE ZIP DATASET FOR STYLEGAN2

!python dataset_tool.py --source=/content/fer2013_256 --dest=/content/datasets/disgust.zip

  0% 0/547 [00:00<?, ?it/s]/content/stylegan2-ada-pytorch/dataset_tool.py:429: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = PIL.Image.fromarray(img, { 1: 'L', 3: 'RGB' }[channels])
100% 547/547 [00:02<00:00, 205.82it/s]


In [ ]:
print(BATCH_SIZE)

8


In [ ]:
## 🧩 SECTION 6 — FINE-TUNE FROM PRETRAINED FFHQ



# Tải pretrained model FFHQ (được NVIDIA phát hành)
!wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-ffhq-256x256.pkl -O ffhq256.pkl

# Chạy fine-tuning
!python train.py \
  --outdir=$PROJECT_DIR/outputs \
  --data=/content/datasets/disgust.zip \
  --resume=/content/drive/MyDrive/stylegan2_fer2013/outputs/00001-disgust-auto1-batch8-noaug-resumecustom/network-snapshot-000020.pkl \
  --snap=5 \
  --cfg=auto \
  --batch=8 \
  --gpus=1 \
  --aug=noaug


Streaming output truncated to the last 5000 lines.
/content/stylegan2-ada-pytorch/torch_utils/ops/conv2d_gradfix.py:55: UserWarning: conv2d_gradfix not supported on PyTorch 2.8.0+cu126. Falling back to torch.nn.functional.conv2d().
  warnings.warn(f'conv2d_gradfix not supported on PyTorch {torch.__version__}. Falling back to torch.nn.functional.conv2d().')
/content/stylegan2-ada-pytorch/torch_utils/ops/conv2d_gradfix.py:55: UserWarning: conv2d_gradfix not supported on PyTorch 2.8.0+cu126. Falling back to torch.nn.functional.conv2d().
  warnings.warn(f'conv2d_gradfix not supported on PyTorch {torch.__version__}. Falling back to torch.nn.functional.conv2d().')
/content/stylegan2-ada-pytorch/torch_utils/ops/conv2d_gradfix.py:55: UserWarning: conv2d_gradfix not supported on PyTorch 2.8.0+cu126. Falling back to torch.nn.functional.conv2d().
  warnings.warn(f'conv2d_gradfix not supported on PyTorch {torch.__version__}. Falling back to torch.nn.functional.conv2d().')
/content/stylegan2-ada-py

In [ ]:
## 🎨 SECTION 7 — GENERATE SYNTHETIC IMAGES

!python generate.py \
--outdir=$PROJECT_DIR/generated/disgust \
--trunc=1 \
--seeds=0-$SAMPLES_TO_GENERATE \
--network=$PROJECT_DIR/outputs/latest.pkl


Loading networks from "/content/drive/MyDrive/stylegan2_fer2013/outputs/latest.pkl"...
Traceback (most recent call last):
  File "/content/stylegan2-ada-pytorch/generate.py", line 127, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
    ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1462, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1383, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1246, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 814, in invoke
    return callback(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/decorators.py", line 